In [1]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

import plotly.express as px

from sklearn.metrics import confusion_matrix
import numpy as np

import plotly.graph_objects as go

import networkx as nx


from operator import itemgetter




/Users/jackohagan/miniforge3/envs/tensorflow/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#import csv of unique pairings 
df = pd.read_csv('couples_network.csv')


In [3]:
##filter value
season = 8

##filter

df = df[df['series'] == season]

In [4]:
##groupe by canddiate and couple
groups = df.groupby(['Candidate','Couple']).agg('count').reset_index().rename(columns={'Unnamed: 0':'weight','Candidate':'source','Couple':'target'})

##connections
sizes = df.groupby('Candidate').count().rename(columns={'Couple':'size'})

In [21]:
##convert to dict for networkx use
sizesdict = sizes['size'].to_dict()

Now we have formatted data, create the network graph. 
We do this by creating nodes (our contestants) and edges (links between contestants)

In [6]:
# Convert your dataframe to graph
G = nx.from_pandas_edgelist(groups, edge_attr=True)

# Generate the layout and set the 'pos' attribute
pos = nx.drawing.layout.spring_layout(G)
pos = nx.fruchterman_reingold_layout(G, k=1, iterations=100)

nx.set_node_attributes(G, pos,'pos')
nx.set_node_attributes(G, sizesdict,'size')


edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

In [7]:
def make_edge(x, y, text, width):
    
    '''Creates a scatter trace for the edge between x's and y's with given width

    Parameters
    ----------
    x    : a tuple of the endpoints' x-coordinates in the form, tuple([x0, x1, None])
    
    y    : a tuple of the endpoints' y-coordinates in the form, tuple([y0, y1, None])
    
    width: the width of the line

    Returns
    -------
    An edge trace that goes between x0 and x1 with specified width.
    '''
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')

In [8]:
# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in G.edges():
    
    if G.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]

        x0, y0 = pos[char_1]
        x1, y1 = pos[char_2]

        text   = char_1 + '--' + char_2 + ': ' + str(G.edges()[edge]['weight'])
        
        trace  = make_edge([x0, x1, None], [y0, y1, None], text,
                           0.3*G.edges()[edge]['weight']**1.75)

        edge_trace.append(trace)

In [9]:
# Make a node trace to visualise
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 15,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node in midsummer, get the position and size and add to the node_trace

node_text = []

for node in G.nodes():
        x, y = pos[node]
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace['marker']['color'] += tuple(['cornflowerblue'])
        node_trace['text'] += tuple(['<b>' + node + '</b>'])
      #  node_trace['hovertemplate'] = tuple(['<b>' + str(G.nodes()[node]['size']) + '</b>'])
        node_trace['marker']['size'] += tuple([5*G.nodes()[node]['size']])

In [10]:
##layout and visualise node
layout = go.Layout(
    title = "Love Island Series {} Recoupling Network Graph".format(season),
    paper_bgcolor="White",
    plot_bgcolor='rgba(0,0,0,0)',
    width=1500,
    height=750,
)


fig = go.Figure(layout = layout)

for trace in edge_trace:
    fig.add_trace(trace)

fig.add_trace(node_trace)

fig.update_layout(showlegend = False)


fig.update_xaxes(showgrid=False, zeroline=False, showticklabels = False)
fig.update_yaxes(showgrid=False, zeroline=False, showticklabels = False)

fig.write_image('visuals/series_{}_graph.png'.format(season))



In [11]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

In [12]:
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

In [13]:
print("Top 5 nodes by degree:")
for d in sorted_degree[:5]:
    print(d)

Top 5 nodes by degree:
('Danica', 6)
('Paige', 4)
('Jay', 4)
('Dami', 3)
('Davide', 3)


In [14]:
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality
closeness_dict = nx.closeness_centrality(G) # Run eigenvector centrality


pagerank_dict = nx.pagerank(G) # Run pagerank centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

nx.set_node_attributes(G, pagerank_dict, 'pagerank')

nx.set_node_attributes(G, closeness_dict, 'closeness')

In [16]:
sorted_closeness = sorted(closeness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 5 nodes by closesness centrality:")
for b in sorted_closeness[:5]:
    print(b)

Top 5 nodes by closesness centrality:
('Danica', 0.3404139433551199)
('Jay', 0.2967711301044635)
('Josh', 0.2967711301044635)
('Luca', 0.28935185185185186)
('Billy', 0.2822944896115628)
